In [18]:
include("functions.jl")

genetique

In [2]:
# Définition des instances de base
@static if !@isdefined(instance_list)
    const instance_list = (
        read_instance("graphs/flat300_26_0.col", 26),
        read_instance("graphs/le450_15c.col", 15),
        read_instance("graphs/dsjc125.1.col", 5),
        read_instance("graphs/dsjc125.9.col", 44),
        read_instance("graphs/dsjc250.1.col", 8),
        read_instance("graphs/dsjc250.9.col", 72),
        read_instance("graphs/dsjc250.5.col", 28),
        read_instance("graphs/dsjc1000.5.col", 86),
        read_instance("graphs/dsjc1000.5.col", 85),
        read_instance("graphs/dsjc1000.5.col", 84))
end

(Instance("flat300_26_0.col", Bool[0 1 … 0 0; 1 0 … 1 1; … ; 0 1 … 0 1; 0 1 … 1 0], 26), Instance("le450_15c.col", Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 1 … 0 0; 0 0 … 0 0], 15), Instance("dsjc125.1.col", Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 5), Instance("dsjc125.9.col", Bool[0 1 … 1 1; 1 0 … 0 1; … ; 1 0 … 0 0; 1 1 … 0 0], 44), Instance("dsjc250.1.col", Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 1 … 0 1; 0 0 … 1 0], 8), Instance("dsjc250.9.col", Bool[0 1 … 1 1; 1 0 … 0 1; … ; 1 0 … 0 1; 1 1 … 1 0], 72), Instance("dsjc250.5.col", Bool[0 1 … 1 0; 1 0 … 1 0; … ; 1 1 … 0 0; 0 0 … 0 0], 28), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 86), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 85), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 84))

# 1) Heuristique gloutonne

L'algorithme glouton choisi:
 - assigne successivement une couleur à chaque nœud
 - les nœuds sont traités dans l'ordre de degré décroissant
 - en choisissant pour chaque nœud la couleur minimisant le nombre de conflits avec ses voisins.

In [3]:
# Comparaison d'une solution aléatoire et d'une solution gloutonne
begin
    instance = instance_list[3]
    aleatoire = sol_alea(instance)

    println(instance.name,"\tk=",instance.k)
    println("SOLUTION ALÉATOIRE:")
    println(aleatoire.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,aleatoire))
    gloutonne = glouton(instance)
    println("\nSOLUTION GLOUTONNE:")
    println(gloutonne.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,gloutonne))
end

dsjc125.1.col	k=5
SOLUTION ALÉATOIRE:


[5, 2, 5, 4, 4, 4, 5, 1, 4, 2, 5, 3, 1, 3, 5, 5, 3, 4, 2, 2, 3, 2, 1, 1, 2, 2, 4, 4, 1, 3, 3, 2, 4, 2, 2, 4, 2, 1, 4, 4, 4, 3, 3, 1, 2, 5, 1, 4, 3, 1, 1, 5, 1, 3, 3, 2, 4, 2, 1, 3, 4, 4, 4, 2, 5, 2, 5, 3, 1, 2, 1, 5, 1, 2, 1, 2, 3, 5, 3, 3, 3, 3, 4, 2, 4, 1, 5, 5, 3, 3, 5, 3, 1, 5, 5, 3, 2, 4, 4, 2, 2, 2, 4, 3, 5, 5, 1, 3, 2, 2, 5, 4, 4, 3, 2, 1, 2, 2, 2, 5, 1, 2, 4, 4, 4]
collisions: 

144



SOLUTION GLOUTONNE:
[3, 2, 2, 4, 2, 5, 4, 3, 4, 3, 2, 5, 3, 3, 2, 4, 4, 4, 3, 5, 5, 5, 5, 4, 5, 2, 3, 3, 2, 5, 4, 3, 4, 4, 3, 5, 4, 2, 4, 5, 5, 4, 2, 2, 5, 4, 3, 5, 3, 2, 2, 2, 4, 3, 4, 4, 4, 5, 5, 2, 2, 2, 5, 3, 3, 5, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 5, 4, 4, 4, 3, 4, 2, 2, 2, 5, 3, 3, 3, 2, 4, 3, 3, 5, 5, 4, 2, 5, 3, 2, 3, 4, 5, 2, 5, 5, 3, 4, 4, 3, 2, 2, 5, 2, 4, 5, 5, 2, 4, 4, 3, 2, 2, 3]
collisions: 47


## Tests numériques de l'heuristique gloutonne

In [4]:
begin
    nsamples = 10
    println("INSTANCE NAME AND k\tMIN CONFL\tMEAN CONFL\tMAX CONFL\tTOTAL TIME\tTIME BEST SOL\tSOLS PER SECOND")
    for instance ∈ instance_list
        conflicts_samples = Vector{Int}(undef,nsamples)
        time_samples = zeros(nsamples)
        for i = 1:nsamples
            solution = glouton(instance)
            start_time = time_ns()
            conflicts = nbr_collision(instance,solution)
            run_time = time_ns()-start_time
            conflicts_samples[i] = conflicts
            time_samples[i] = run_time*1e-9
        end
        println(
            instance.name," k=",instance.k,"\t",
            minimum(conflicts_samples),"\t",
            sum(conflicts_samples)/nsamples,"\t",
            maximum(conflicts_samples),"\t",
            sum(time_samples),"\t",
            time_samples[argmin(conflicts_samples)],"\t",
            nsamples/sum(time_samples)
        ) # solutions per second irrelevant, isn't it? (since there is no local search loop)
    end
end

INSTANCE NAME AND k	MIN CONFL	MEAN CONFL	MAX CONFL	TOTAL TIME	TIME BEST SOL	SOLS PER SECOND


flat300_26_0.col k=26	226	226.0	226	0.0027348000000000003	0.00027330000000000003	3656.5745209887373
le450_15c.col k=15	362	362.0	362	0.0033281	0.0003334	3004.717406327935
dsjc125.1.col k=5	47	47.0	47	0.0002097	2.11e-5	47687.17215069146
dsjc125.9.col k=44	21	21.0	21	0.0007890000000000001	8.970000000000001e-5	12674.271229404309


dsjc250.1.col k=8	88	88.0	88	0.0016938	0.00019910000000000001	5903.884756169559
dsjc250.9.col k=72	61	61.0	61	0.0011351	0.00013230000000000002	8809.796493700995
dsjc250.5.col k=28	85	85.0	85	0.0019194000000000004	0.00019580000000000002	5209.961446285297


dsjc1000.5.col k=86	378	378.0	378	0.0305526	0.003039	327.3043865333883


dsjc1000.5.col k=85	398	398.0	398	0.030836000000000002	0.0030570000000000003	324.29627707873914


dsjc1000.5.col k=84	412	412.0	412	0.030500700000000006	0.0030372000000000003	327.86132777280517


# 2) Structure de voisinage

On choisit d'utiliser un opérateur de voisinage simple:
 - Appliquer la couleur $i$ au nœud $j$ choisis de manière à réduire le plus possible le nombre de conflits par rapport à la solution courante.
   - On choisit le premier couple $(i,j)$ occasionnant la plus forte réduction de conflits.
   - Pour cela, comme suggéré par Daniel Porumbel, on maintient une matrice des conflits stockant pour chaque nœud $j$ et chaque couleur $i$ le nombre de conflits en ce nœud $j$ si l'on lui applique la couleur $i$.

On obtient donc une structure de voisinage qui contient toutes les solutions dont un unique nœud diffère par sa couleur de la solution d'origine.

In [5]:
# Test de descente locale simple
begin
    instance = instance_list[3]
    solution = glouton(instance)
    simple_neighbor(instance,solution)
    print(solution.obj)
    for t = 1:25
        simple_neighbor(instance,solution)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

43t: 1	conflits: 40
t: 2	conflits: 38
t: 3	conflits: 36
t: 4	conflits: 34
t: 5	conflits: 32
t: 6	conflits: 30
t: 7	conflits: 28
t: 8	conflits: 27
t: 9	conflits: 26
t: 10	conflits: 25
t: 11	conflits: 24
t: 12	conflits: 23
t: 13	conflits: 22
t: 14	conflits: 21
t: 15	conflits: 20
t: 16	conflits: 19
t: 17	conflits: 18
t: 18	conflits: 17
t: 19	conflits: 16
t: 20	conflits: 15
t: 21	conflits: 15
t: 22	conflits: 15
t: 23	conflits: 15
t: 24	conflits: 15
t: 25	conflits: 15


# 3) Métaheuristique proposée

On choisit d'écrire un algorithme mémétique:

## 0 - Règle de sélection

On utilise la règle de la roue de la fortune, en utilisant comme poids l'objectif dans l'opérateur d'élimination et l'inverse de l'objectif dans la fonction de reproduction.

## 1 - Fonction de reproduction

On génère λ enfants, chacun à partir de deux parents sélectionnés avec la règle ci-dessus, en leur appliquant l'opérateur de croisement.\
Afin de garantir la diversité des solutions, si l'enfant est trop proche d'une solution existante (plus proche que le seuil de rejet, voir opérateur d'élimination), il est rejeté.

### Opérateur de croisement

Comme suggéré par Daniel Porumbel, on choisit un opérateur de croisement qui copie des classes de couleurs entières des parents dans l'enfant (sur les nœuds où l'enfant n'a pas encore de couleur définie), dans l'ordre décroissant de taille de la classe, indifféremment de leur origine parentale.

## 2 - Opérateur d'intensification

On effectue une recherche locale simple, en utilisant la structure de voisinage simple `simple_neighbor` citée ci-dessus.\
On interrompt la recherche locale dès que l'on atteint un minimum local ou un plateau (i.e. si la valeur de l'objectif n'a pas changé). Dans tous les cas, la recherche locale se termine si l'on atteint le nombre maximal d'itérations autorisé.

## 3 - Fonction d'élimination

Comme proposé par Daniel Porumbel, on calcule la distance entre chaque individu de la population.
 - Si la distance minimale est inférieure au seuil de rejet, on élimine la solution de moins bon objectif.
 - Sinon, on utilise la règle de sélection pour obtenir une solution, on calcule son voisin le plus proche, et on élimine celle de moins bon objectif.

### Calcul de distance entre deux solutions

Comme Daniel Porumbel l'a fait remarquer, la distance entre deux solutions n'est pas simplement le nombre de couleurs différentes, car la solution est définie à une permutation des couleurs près. Pour calculer la distance, il est donc nécessaire de déterminer la permutation minimisant le nombre de couleurs différentes entre deux solutions, donc maximisant le nombre de couleurs identiques:

$\displaystyle{\max_{\text{perm}} \sum_{c} S(c,\text{perm}(c))}$

Où $S$ est la matrice de similarité, i.e. $S(a,b)$ est le nombre de nœuds $i$ pour lesquels $x_1(i)=a$ et $x_2(i)=b$.

Le calcul de la meilleure permutation est un problème NP-complet, ce qui exclut l'utilisation d'un algorithme exact pour la déterminer. On écrit donc une heuristique:

Les cases de $S$ contenant les plus grandes valeurs ont de grandes chances d'être atteintes par la permutation. On définit donc la permutation comme suit:
 - $(a,b) = \argmax(S)$
 - $\text{perm}(b) = a$
 - vider la colonne $b$ (correspondant à $\text{perm}(b) = u$, car perm est une fonction) ainsi que la ligne $a$ (correspondant à $\text{perm}(v) = a$, car perm est injective)
 - recommencer

Cette heuristique a toutes les chances de fournir la permutation optimale si les solutions sont effectivement proches, ce qui donne une matrice $S$ avec une case de valeur nettement plus grande que les autres de sa ligne/colonne, donc détectable par l'heuristique. Cela tombe bien, car il est important d'avoir une bonne estimation de la distance pour des solutions proches, alors que pour des solutions éloignées, cela est moins critique.

Test: Tabou

In [6]:
# Test de descente locale avec tabou
begin
    instance = instance_list[3]
    solution = glouton(instance)

    for t = 1:100
        tabu_neighbor(instance,solution,10,t)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

t: 1	conflits: 43
t: 2	conflits: 40
t: 3	conflits: 38
t: 4	conflits: 36
t: 5	conflits: 34
t: 6	conflits: 32
t: 7	conflits: 30
t: 8	conflits: 28
t: 9	conflits: 27
t: 10	conflits: 26
t: 11	conflits: 25
t: 12	conflits: 24
t: 13	conflits: 23
t: 14	conflits: 22
t: 15	conflits: 21
t: 16	conflits: 20
t: 17	conflits: 19
t: 18	conflits: 18
t: 19	conflits: 17
t: 20	conflits: 16
t: 21	conflits: 15
t: 22	conflits: 15
t: 23	conflits: 15
t: 24	conflits: 15
t: 25	conflits: 15
t: 26	conflits: 15
t: 27	conflits: 15
t: 28	conflits: 15
t: 29	conflits: 15
t: 30	conflits: 15
t: 31	conflits: 15
t: 32	conflits: 15
t: 33	conflits: 15
t: 34	conflits: 15
t: 35	conflits: 15
t: 36	conflits: 15
t: 37	conflits: 15
t: 38	conflits: 15
t: 39	conflits: 15
t: 40	conflits: 15
t: 41	conflits: 15
t: 42	conflits: 15
t: 43	conflits: 15
t: 44	conflits: 15
t: 45	conflits: 15
t: 46	conflits: 15
t: 47	conflits: 15
t: 48	conflits: 15
t: 49	conflits: 15
t: 50	conflits: 15
t: 51	conflits: 15
t: 52	conflits: 15
t: 53	conflits: 15
t:

Algo mémétique

In [19]:
begin
    start_time = time()
    instance = instance_list[3]
    instopt = genetique(instance,25,25,10,10,80) 
    print("Nombre de collisions : ",instopt.obj, "\n")
    print("temps d'execution : ",trunc(100*(time()-start_time))/100,"s")
end

MethodError: MethodError: no method matching genetique(::Instance, ::Int64, ::Int64, ::Int64, ::Int64, ::Int64)

Closest candidates are:
  genetique(::Instance, ::Int64, ::Int64, ::Int64, ::Int64)
   @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:313
  genetique(::Instance, !Matched::Vector{Solution}, ::Int64, ::Int64, ::Int64, ::Int64)
   @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:319


tests

Cellule d'ouverture des streams, à activer avec précaution

In [8]:
iotemps = open("temps_d_executions", "w")
ioobj = open("val_fonc-obj","w")

write(iotemps, "instance:, ", "graphs/flat300_26_0.col k=26, ")
write(iotemps,"graphs/le450_15c.col k=15, ")
write(iotemps,"graphs/dsjc125.1.col k=5, ")
write(iotemps,"graphs/dsjc125.9.col k=44, ")
write(iotemps,"graphs/dsjc250.1.col k=8, ")
write(iotemps,"graphs/dsjc250.9.col k=72, ")
write(iotemps,"graphs/dsjc250.5.col k=28, ")
write(iotemps,"graphs/dsjc1000.5.col k=86, ")
write(iotemps,"graphs/dsjc1000.5.col k=85, ")
write(iotemps,"graphs/dsjc1000.5.col k=84, \n")

write(ioobj, "instance:, ", "graphs/flat300_26_0.col k=26, ")
write(ioobj,"graphs/le450_15c.col k=15, ")
write(ioobj,"graphs/dsjc125.1.col k=5, ")
write(ioobj,"graphs/dsjc125.9.col k=44, ")
write(ioobj,"graphs/dsjc250.1.col k=8, ")
write(ioobj,"graphs/dsjc250.9.col k=72, ")
write(ioobj,"graphs/dsjc250.5.col k=28, ")
write(ioobj,"graphs/dsjc1000.5.col k=86, ")
write(ioobj,"graphs/dsjc1000.5.col k=85, ")
write(ioobj,"graphs/dsjc1000.5.col k=84, \n")

29

In [9]:
Test glouton

ErrorException: syntax: extra token "glouton" after end of expression

In [10]:
begin
    write(ioobj, "Glouton, ")
    write(iotemps, "Glouton, ")
    for instance in instance_list
        start_time = time()
        solution = glouton(instance)
        print(solution.obj)
        write(ioobj, repr(nbr_collision(instance,solution)), ", ")
        write(iotemps, repr(trunc(100*(time()-start_time))/100),"s, ")
    end
    write(iotemps,"\n")
    write(ioobj,"\n")
end

90000

2025001562515625625006250062500

10000001000000

1000000

1

Je relance des tests avec d'autres paramètres, celui avec moins d'nefant était beaucoup plus rapide sans grosse parte de performance, donc je vais réduire le nombre d'enfant pour tous et voir comment il se comporte (Je le fixe à 10 pour tous) j'essaye de fiar monter MATit aussi

In [17]:
begin
    write(ioobj, "Genetique 25 k/2 10 100 100, ")
    write(iotemps, "Genetique 25 k/2 10 100 100, ")
    for instance in instance_list
        start_time = time()
        solution = genetique(instance,25,div(instance.k,2),10,100,100)
        write(ioobj, repr(solution.obj), ", ")
        write(iotemps, repr(trunc(100*(time()-start_time))/100),"s, ")
    end
    write(iotemps,"\n")
    write(ioobj,"\n")
end

MethodError: MethodError: no method matching genetique(::Instance, ::Int64, ::Int64, ::Int64, ::Int64, ::Int64)

Closest candidates are:
  genetique(::Instance, ::Int64, ::Int64, ::Int64, ::Int64)
   @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:313
  genetique(::Instance, !Matched::Vector{Solution}, ::Int64, ::Int64, ::Int64, ::Int64)
   @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:319


Test avec un plus grand localMAXIT

In [12]:
begin
    write(ioobj, "Genetique 25 k 10 100 100, ")
    write(iotemps, "Genetique 25 10 10 100 100, ")
    for instance in instance_list
        start_time = time()
        solution = genetique(instance,25,instance.k,10,100,100)
        write(ioobj, repr(solution.obj), ", ")
        write(iotemps, repr(trunc(100*(time()-start_time))/100),"s, ")
    end
    write(iotemps,"\n")
    write(ioobj,"\n")
end

MethodError: MethodError: no method matching genetique(::Instance, ::Int64, ::Int64, ::Int64, ::Int64, ::Int64)

Closest candidates are:
  genetique(::Instance, ::Int64, ::Int64, ::Int64, ::Int64)
   @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:313
  genetique(::Instance, !Matched::Vector{Solution}, ::Int64, ::Int64, ::Int64, ::Int64)
   @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:319


Tests avec plus de population

In [13]:
begin
    write(ioobj, "Genetique 50 k/2 10 100 100, ")
    write(iotemps, "Genetique 50 k/2 10 100 100, ")
    for instance in instance_list
        start_time = time()
        solution = genetique(instance,50,instance.k/2,10,100,100)
        write(ioobj, repr(solution.obj), ", ")
        write(iotemps, repr(trunc(100*(time()-start_time))/100),"s, ")
    end
    write(iotemps,"\n")
    write(ioobj,"\n")
end

MethodError: MethodError: no method matching genetique(::Instance, ::Int64, ::Float64, ::Int64, ::Int64, ::Int64)

Closest candidates are:
  genetique(::Instance, ::Int64, !Matched::Int64, ::Int64, ::Int64)
   @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:313
  genetique(::Instance, !Matched::Vector{Solution}, !Matched::Int64, ::Int64, ::Int64, ::Int64)
   @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:319


Test avec plus de distance entre les individuts

In [14]:
begin
    write(ioobj, "Genetique 10 10 k/2 100 100, ")
    write(iotemps, "Genetique 10 10 k/2 100 100, ")
    for instance in instance_list
        start_time = time()
        solution = genetique(instance,10,10,instance.k/2,100,100)
        write(ioobj, repr(solution.obj), ", ")
        write(iotemps, repr(trunc(100*(time()-start_time))/100),"s, ")
    end
    write(iotemps,"\n")
    write(ioobj,"\n")
end

MethodError: MethodError: no method matching genetique(::Instance, ::Int64, ::Int64, ::Float64, ::Int64, ::Int64)

Closest candidates are:
  genetique(::Instance, ::Int64, ::Int64, !Matched::Int64, ::Int64)
   @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:313
  genetique(::Instance, !Matched::Vector{Solution}, ::Int64, !Matched::Int64, ::Int64, ::Int64)
   @ Main c:\Users\franc\OneDrive\Bureau\Julia par oim\Proj-meta\functions.jl:319


Cellule de fermeture des streams, à activer une fois que tout est écrit

In [15]:
close(iotemps)
close(ioobj)

Les tests effectués indiquent que modifier certains paramètres est très coûteux en temps pour une amélioration assez faible de la fonction objective. Je vais donc détailler les choix de paramètres que je préconnise :

Pop size : ne pas avoir trop haut : 10 semble raisonnable

Nbrchildren : paramètre très couteux à avoir haut, à garder assez faible, 10 semble raisonnable

DistTHR : paramètre à garder imperativement faible pour les instances de petites taille, sinon le faire grandir n'est pas très coûteux, Je pense que l'avoir à 3 ou 5 pour les petites instances sinon l'avoir de l'ordre de grandeur de k/2 ne me semble pas absurde

Local Maxit : parametre peut coûteux à gerder élévé : 200 c'est pas ma je pense, de toutes façon c'est plutôt un garde fou qu'autre chose il me semble

Maxit : nombre d'itération de l'algo, c'est chouette quand on peut l'avoir haut

Autres modifications qui me semblent utiles :
Dans beaucoup de cas l'algo glouton est meilleur que le mémétique, c'est problématique, il serait surment souhaitable d'initialiser l'algo mémétique avec dans la population la solution de l'algo glouton.



